In [3]:
import os 
import speech_recognition as sr 
from googletrans import Translator
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM
from flask import Flask, request, jsonify
from flask_cors import CORS
from pydub import AudioSegment  
from transformers import pipeline
import tkinter as tk
from tkinter import filedialog
from moviepy.editor import VideoFileClip
import easygui
import shutil

In [2]:
def select_and_store_file():
    root = tk.Tk()
    root.withdraw()  

    file_path = filedialog.askopenfilename(filetypes=[("Video and Audio Files", "*.mp3 *.mp4 *.wav *.flac *.avi *.mkv")])

    if file_path:
        with open(file_path, 'rb') as file: 
            print(f'Selected file: {file_path}')
    else:
        print("No file selected.")

In [3]:
def select_and_store_file():
    file_path = easygui.fileopenbox(filetypes=["*.mp3 *.mp4 *.wav *.flac *.avi *.mkv"])

    if file_path:
        # Determine the file name from the chosen file path
        file_name = file_path.split("/")[-1]

        # Specify the destination path in the current directory
        destination_path = f"./{file_name}"

        try:
            # Copy the selected file to the current directory
            shutil.copyfile(file_path, destination_path)

            print(f'Selected file copied to current directory: {destination_path}')
        except Exception as e:
            print(f"Error copying the file: {str(e)}")
    else:
        print("No file selected.")

if __name__ == "__main__":
    select_and_store_file()
